In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass

@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 48      # e.g. GPT-2 XL
    n_head: int = 25
    n_embd: int = 1600

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)

        self.register_buffer(
            "bias",
            torch.tril(torch.ones(config.block_size, config.block_size))
            .view(1, 1, config.block_size, config.block_size)
        )

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(C, dim=2)

        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)

        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y


class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


class GPT(nn.Module):
    """
    A GPT-like model that only stores the hidden state *after*
    'skip_up_to - 1' layers (the "Block k" state).

    For skipping:
      - if we detect a copy scenario, we load the cached partial
        hidden state from t_matched, run the last layers only.
      - else, we run all layers and store the partial state.
    """

    def __init__(self, config, skip_up_to=43):
        super().__init__()
        self.config = config
        self.skip_up_to = skip_up_to   # number of layers to skip
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # cache_partial: only store the hidden state after skip_up_to - 1 layers
        # so cache_partial[(batch_idx, pos)] = Tensor shape (B, T, n_embd)
        self.cache_partial = {}

    def forward(self, input_ids, pos_matched=None):
        B, T = input_ids.shape
        device = input_ids.device

        # Basic embed
        pos = torch.arange(0, T, dtype=torch.long, device=device)
        full_x = self.transformer.wte(input_ids) + self.transformer.wpe(pos)

        if pos_matched is None:
            # Normal forward for all T tokens from layer 0..n_layer
            x = full_x
            for block in self.transformer.h:
                x = block(x)

        else:
            # "Copy" scenario
            # 1) separate the sequence into first T-1 tokens vs the newly added token
            x_trunc = full_x[:, :-1, :]  # shape = (1, T-1, n_embd)

            # 2) compute from layer 0..skip_up_to on the truncated x
            for layer_idx in range(self.skip_up_to):
                x_trunc = self.transformer.h[layer_idx](x_trunc)

            # 3) get matched hidden from x_trunc for the new token
            #    note that pos_matched must be < T-1, so we can do x_trunc[:, pos_matched, :]
            matched_hid = x_trunc[:, pos_matched, :] # shape (1, n_embd)            
            matched_hid = matched_hid.unsqueeze(1) # => shape (1, 1, n_embd)

            # 4) cat matched hidden to x_trunc => new shape (1, T, n_embd)
            x = torch.cat([x_trunc, matched_hid], dim=1)

            # 5) continue from layer skip_up_to+1..end on the full (1, T, n_embd)
            for layer_idx in range(self.skip_up_to, self.config.n_layer):
                x = self.transformer.h[layer_idx](x)

        # final layer norm + logits
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)  # shape (B, T, vocab_size)
        return logits

    @classmethod
    def from_pretrained(cls, model_type, skip_up_to=43):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config, skip_up_to=skip_up_to)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_top_k(logits, top_k=5):
    """
    logits: (B, T, vocab_size)
    Returns a list of top-k token IDs for the last position, e.g. [id1, id2,...].
    """
    last_logits = logits[:, -1, :]       # shape (B, vocab_size)
    probs = torch.softmax(last_logits, dim=-1)
    top_vals, top_indices = probs.topk(top_k, dim=-1)
    # top_indices is shape (B, top_k). For B=1, we do top_indices[0].tolist().
    return top_indices[0].tolist()

def detect_ngram_copy(seq_ids: torch.Tensor, n=3, skip_up_to=43):
    """
    Minimal function that tries to find n-gram copy scenario
    (just a placeholder – adapt to your real logic)
    """
    T = seq_ids.size(1)  # shape (B=1, T)
    if T < n:
        return None, None
    # 1) last token
    last_token = seq_ids[0, -1].item()
    # 2) find earlier positions of last_token
    possible_pos = (seq_ids[0, :-1] == last_token).nonzero().view(-1)
    if possible_pos.numel() == 0:
        return None, None
    # 3) check (n-1) context
    n_minus_1 = n - 1
    context_needed = seq_ids[0, -(n_minus_1+1):-1]  # last n-1 tokens
    matched_pos = None
    for pos in reversed(possible_pos):
        if pos >= n_minus_1:
            candidate = seq_ids[0, pos-n_minus_1:pos]
            if torch.all(candidate == context_needed):
                matched_pos = pos.item()
                break
    if matched_pos is None:
        return None, None
    else:
        return matched_pos, skip_up_to

In [ ]:
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return  intersection / union


def get_acc(info_lst):
    jcc_ult = []
    acc_ult = []

    for data in info_lst:
        acc_lst = []
        jc_lst = []
        for step in data.keys():
            copy = data[step]['copy']
            original = data[step]['original']

            jaccard_score = jaccard_similarity(copy, original)
            jc_lst.append(jaccard_score)

            acc_score = 1 if copy[0] == original[0] else 0
            acc_lst.append(acc_score)

        jcc_ult.append(jc_lst)
        acc_ult.append(acc_lst)
    
    def cal_avg(lsts):
        avg_lst = []
        for lst in lsts:
            avg_lst.append(sum(lst) / len(lst))
        return sum(avg_lst) / len(avg_lst)

    avg_jcc = cal_avg(jcc_ult)
    avg_acc = cal_avg(acc_ult)
    print("Average Jaccard Similarity: ", avg_jcc)
    print("Average Accuracy: ", avg_acc)

    return avg_jcc, avg_acc

In [ ]:
from transformers import GPT2Tokenizer

model_name = 'gpt2-xl'
skip_up_to = 5

device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

model_copy = GPT.from_pretrained(model_name, skip_up_to=skip_up_to)
model_copy = model_copy.to(device1)
model_copy.eval()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = GPT.from_pretrained(model_name, skip_up_to=skip_up_to)
model = model.to(device)
model.eval()

Code Repair

In [3]:
import os

# Specify the folder containing the .py files
folder_path = "QuixBugs/python_programs"  # Replace with the actual path to your folder

# Initialize an empty list to store file contents
file_contents = []

# Iterate through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".py"):  # Check if the file is a .py file
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, "r") as file:
            content = file.read()  # Read the file content
            content = content.replace("    ", "\t")
            file_contents.append(content)  # Add the content to the list

Test model match with Hugging Face

In [6]:
import time
import torch
import random
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm
from collections import defaultdict

def mini_acc(dict_pred_info):
    acc_lst = []
    for step in dict_pred_info.keys():
        copy = dict_pred_info[step]['copy']
        original = dict_pred_info[step]['original']

        acc_score = 1 if copy[0] == original[0] else 0
        acc_lst.append(acc_score)
    return sum(acc_lst) / len(acc_lst)

def ngram(n, model_copy, model, skip_up_to, max_steps, extra_steps, k, file_contents):
    model_copy.eval()
    model.eval()
    model.skip_up_to = 0

    print("n-gram: ", n)
    print("Previous skip layers: ", model_copy.skip_up_to)
    model_copy.skip_up_to = skip_up_to
    print("New skip layers: ", model_copy.skip_up_to)

    info_lst = []
    failed_lst = []
    total_failed = 0

    num_matched = 0
    for code in tqdm(file_contents):
        prompt = f"Correct the following code:\n{code}\nCorrected code: def"
        
        code_ids = tokenizer.encode(code, return_tensors='pt')
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        steps = extra_steps+code_ids.size(1)
        if steps > max_steps:
            steps = max_steps

        dict_pred_info = defaultdict(dict)

        # Copy model generation
        copy_ids = input_ids.clone().to(device1)
        for step_i in range(steps):
            t0 = time.time()

            # detect copy scenario
            t_matched, skip_up_to = detect_ngram_copy(copy_ids, n=n, skip_up_to=skip_up_to)
            if t_matched is not None:
                num_matched += 1

            with torch.no_grad():
                # forward pass (copy-mech model)
                logits = model_copy(
                    input_ids=copy_ids,
                    pos_matched=t_matched,
                )

                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                topk_probs, topk_indices = torch.topk(probs, k, dim=-1)                
                # get the highest prob token
                next_token = topk_indices[:, 0].unsqueeze(1)
                # append the token to the sequence
                copy_ids = torch.cat([copy_ids, next_token], dim=1)

            elapsed_copy = time.time() - t0

            # store info
            dict_pred_info[step_i]['copy'] = topk_indices[0].tolist()
            dict_pred_info[step_i]['copy_time'] = elapsed_copy
        model_copy.cache_partial.clear()
        
        # 2) Original model generation
        t_matched, skip_up_to = (None, None)
        original_ids = input_ids.clone().to(device)
        for step_i in range(steps):
            t0 = time.time()

            with torch.no_grad():
                # forward pass
                logits = model(
                    input_ids=original_ids,
                )
                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                topk_probs, topk_indices = torch.topk(probs, k, dim=-1)
                # get the highest prob token
                next_token = topk_indices[:, 0].unsqueeze(1)
                # append the token to the sequence
                original_ids = torch.cat([original_ids, next_token], dim=1)
                
            elapsed_orig = time.time() - t0

            # store info
            dict_pred_info[step_i]['original'] = topk_indices[0].tolist()
            dict_pred_info[step_i]['original_time'] = elapsed_orig

        model.cache_partial.clear()
        current_acc = mini_acc(dict_pred_info)
        
        if current_acc < 1:
            print("Failed accuracy: ", current_acc)
            failed = {}
            failed['code'] = code
            failed['predicted'] = tokenizer.decode(copy_ids[0])
            failed['original'] = tokenizer.decode(original_ids[0])
            failed['info'] = dict_pred_info
            failed['acc'] = current_acc
            failed_lst.append(failed)    
            total_failed += 1

        info_lst.append(dict_pred_info)

    print("Number of matched tokens: ", num_matched)
    print("Avg matched tokens per program: ", num_matched/len(file_contents))
    print("Total failed programs: ", total_failed)
    jcc, acc = get_acc(info_lst)
    return info_lst, failed_lst, num_matched, jcc, acc

In [7]:
seed = 5
extra_steps = 30
max_steps = 1024
k=100
seed_everything(seed)

ns = [5]
skip_up_to = [5]
info_lst = {}
for n in ns:
    info_lst[n] = {}
    for skip in skip_up_to:
        info_lst[n][skip] = {}
        outputs = ngram(n, model_copy, model, skip, max_steps, extra_steps, k, file_contents)
        info_lst[n][skip]['info'] = outputs[0]
        info_lst[n][skip]['failed'] = outputs[1]
        info_lst[n][skip]['num_matched'] = outputs[2]
        info_lst[n][skip]['jcc'] = outputs[3]
        info_lst[n][skip]['acc'] = outputs[4]
        print("-----------------------------------")

n-gram:  5
Previous skip layers:  5
New skip layers:  5


  0%|          | 0/40 [00:00<?, ?it/s]

  2%|▎         | 1/40 [00:28<18:28, 28.42s/it]

Failed accuracy:  0.06467661691542288


  5%|▌         | 2/40 [00:43<13:06, 20.69s/it]

...

Failed accuracy:  0.34579439252336447


 90%|█████████ | 36/40 [10:06<01:04, 16.12s/it]

Failed accuracy:  0.5


100%|██████████| 40/40 [10:36<00:00, 15.91s/it]

Number of matched tokens:  3241
Avg matched tokens per program:  81.025
Total failed programs:  19
Average Jaccard Similarity:  0.64257039534966
Average Accuracy:  0.6435183067177357
-----------------------------------


English Insertion

In [ ]:
from datasets import load_from_disk
from tqdm import tqdm

# Load the dataset from disk
subset = load_from_disk("english_insertions")
prompt_list = []

base_sents = subset['train']['base_sentence'][:1000]
phrases = subset['train']['phrase'][:1000]
edited_sents = subset['train']['edited_sentence'][:1000]

import gc
del subset
gc.collect()